# Principle Component Analysis
____________________________________
* Aayush Malde (1911090)
* Aditya Malwade (1911091)
* Rahul Panchal (1911097)


In [10]:
!pip install ipython-autotime

In [11]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

import seaborn as sns

from bokeh.plotting import figure, output_file, show
from bokeh.layouts import row



import sklearn
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split,  cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.decomposition import PCA, IncrementalPCA

import plotly.express as px

from sklearn.decomposition import PCA

In [24]:
%load_ext autotime
import warnings
warnings.filterwarnings('ignore')

time: 136 µs (started: 2022-04-19 16:59:04 +00:00)


# Preparing DATA

In [12]:
mnist_train = pd.read_csv('/content/sample_data/mnist_train_small.csv',header=None)
mnist_test = pd.read_csv('/content/sample_data/mnist_test.csv',header=None)

In [13]:
mnist_train.head(10)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
train_labels = mnist_train[0]
train_labels.head(10)

0    6
1    5
2    7
3    9
4    5
5    2
6    0
7    6
8    5
9    5
Name: 0, dtype: int64

In [15]:
test_labels = mnist_test[0]
test_labels.head(10)

0    7
1    2
2    1
3    0
4    4
5    1
6    4
7    9
8    5
9    9
Name: 0, dtype: int64

In [16]:
mnist_train.drop(columns=mnist_train.columns[0],inplace=True)
mnist_train.head(10)

,1,2,3,4,5,6,7,8,9,10,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
mnist_test.drop(columns=mnist_test.columns[0],inplace=True)
mnist_test.head(10)

,1,2,3,4,5,6,7,8,9,10,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# NON PCA

In [25]:
# defining Random Forest Classifier Model
rfc = RandomForestClassifier() 
rfc.fit(mnist_train, train_labels)
rfc_pred = rfc.predict(mnist_test)
print('Accuracy of Random Forest : ', metrics.accuracy_score(test_labels, rfc_pred))

Accuracy of Random Forest :  0.9599
time: 12 s (started: 2022-04-19 16:59:14 +00:00)


In [26]:
# defining Logistic Regression Classifier Model
logisticRegr = LogisticRegression()
logisticRegr.fit(mnist_train, train_labels)
predictions  = logisticRegr.predict(mnist_test)
print('Accuracy of Logistic Regression : ', metrics.accuracy_score(test_labels, predictions))

Accuracy of Logistic Regression :  0.9122
time: 8.77 s (started: 2022-04-19 16:59:26 +00:00)


# PCA

In [27]:
#PCA on the training and testing datasets
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(mnist_train)

train_img = scaler.transform(mnist_train)
test_img = scaler.transform(mnist_test)

pca = PCA(.95)

pca.fit(train_img)

train_img = pca.transform(train_img)
test_img = pca.transform(test_img)

time: 3.17 s (started: 2022-04-19 16:59:35 +00:00)


In [28]:
pd.DataFrame(train_img)

,0,1,2,3,4,5,6,7,8,9,...,297,298,299,300,301,302,303,304,305,306
0,10.820653,-0.055053,-2.687310,4.602921,11.268307,-0.536771,-1.622079,-4.644912,2.891247,-2.106295,...,0.343699,-1.330739,-0.323928,0.369224,-0.276479,0.418771,0.134274,0.041291,-0.650693,-0.823855
1,2.002758,1.137359,0.807016,-7.342817,-2.164604,-5.277946,0.729776,-2.772978,2.238423,2.837908,...,0.244449,-0.659919,-0.227053,-0.439335,-0.512784,0.449603,0.167553,-0.253671,-0.244638,-0.445931
2,3.720519,-1.708229,-10.601088,0.125450,0.772593,-4.651477,1.219495,1.852789,0.291810,3.320325,...,-0.163718,0.140699,0.315532,0.162407,-0.026503,-0.142103,0.464724,0.769757,0.563387,0.530187
3,2.713128,9.445983,-6.676673,0.588088,-3.336508,-3.684184,4.548247,-0.935292,0.030126,-0.484301,...,0.453399,-0.024516,0.445324,0.576018,-0.096971,0.160936,-0.226561,0.283900,0.390357,0.270620
4,3.884133,4.498352,3.153827,-5.044817,-0.495475,-5.764537,0.604561,-3.114133,0.730742,1.729122,...,-0.285970,0.308574,0.445696,0.107290,-0.505914,-0.060210,-0.691749,-0.512887,0.609931,0.041038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,10.008743,10.160619,2.076156,-3.465100,4.801173,0.107474,-2.245169,-4.305408,1.431429,-0.699089,...,0.789951,0.389970,-0.548099,-0.151208,-0.037341,0.497972,0.366738,-0.354168,0.065960,0.121390
19996,-7.659516,-0.663501,2.085924,1.593653,4.166492,-4.415976,-0.043244,2.126438,0.865716,-0.194594,...,-0.354913,0.027212,0.216926,-0.402322,-0.091876,0.213345,0.080358,-0.275288,0.063690,0.046872
19997,8.547860,-10.938702,-1.541649,-0.128295,0.319450,6.929045,6.441481,1.864523,1.060482,2.054823,...,-0.165130,-0.494793,0.096582,1.192492,-0.833001,0.813217,-0.135073,-0.898087,0.442498,-1.035247
19998,-5.461561,-4.168869,-4.708746,-0.455199,0.499132,3.637600,-3.662189,0.963211,-1.115097,2.618795,...,-0.267920,-0.139423,0.778554,-0.309727,-0.192406,0.655787,-0.167162,0.079853,0.252958,0.322284


time: 38.4 ms (started: 2022-04-19 16:59:38 +00:00)


In [29]:
#Random Forest after using PCA
rfc = RandomForestClassifier() 
rfc.fit(train_img, train_labels)
rfc_pred = rfc.predict(test_img)
print('Accuracy of Random Forest : ', metrics.accuracy_score(test_labels, rfc_pred))

Accuracy of Random Forest :  0.9232
time: 36.6 s (started: 2022-04-19 16:59:38 +00:00)


In [30]:
#logistic regression after using PCA
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(train_img, train_labels)
# logisticRegr.score(test_img, test_labels)
predictions  = logisticRegr.predict(test_img)
print('Accuracy of Logistic Regression : ', metrics.accuracy_score(test_labels, predictions))

Accuracy of Logistic Regression :  0.9158
time: 4.27 s (started: 2022-04-19 17:00:14 +00:00)


# Randomized PCA

In [31]:
from sklearn.decomposition import PCA as RandomizedPCA

time: 883 µs (started: 2022-04-19 17:02:39 +00:00)


In [32]:
#PCA on the training and testing datasets
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(mnist_train)

train_img = scaler.transform(mnist_train)
test_img = scaler.transform(mnist_test)

pca = RandomizedPCA(n_components=300, svd_solver='randomized', whiten=True).fit(train_img)

train_img = pca.transform(train_img)
test_img = pca.transform(test_img)

time: 7.89 s (started: 2022-04-19 17:02:40 +00:00)


In [33]:
#Random Forest after using PCA
rfc = RandomForestClassifier() 
rfc.fit(train_img, train_labels)
rfc_pred = rfc.predict(test_img)
print('Accuracy of Random Forest : ', metrics.accuracy_score(test_labels, rfc_pred))

Accuracy of Random Forest :  0.9253
time: 36.9 s (started: 2022-04-19 17:02:48 +00:00)


In [34]:
#logistic regression after using PCA
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(train_img, train_labels)
# logisticRegr.score(test_img, test_labels)
predictions  = logisticRegr.predict(test_img)
print('Accuracy of Logistic Regression : ', metrics.accuracy_score(test_labels, predictions))

Accuracy of Logistic Regression :  0.9143
time: 4.4 s (started: 2022-04-19 17:03:25 +00:00)


# Incremental PCA

In [35]:
#PCA on the training and testing datasets
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(mnist_train)

train_img = scaler.transform(mnist_train)
test_img = scaler.transform(mnist_test)

ipca = IncrementalPCA(n_components = 300).fit(train_img)

train_img = ipca.transform(train_img)
test_img = ipca.transform(test_img)

time: 4.42 s (started: 2022-04-19 17:03:29 +00:00)


In [36]:
#Random Forest after using PCA
rfc = RandomForestClassifier() 
rfc.fit(train_img, train_labels)
rfc_pred = rfc.predict(test_img)
print('Accuracy of Random Forest : ', metrics.accuracy_score(test_labels, rfc_pred))

Accuracy of Random Forest :  0.9242
time: 39.7 s (started: 2022-04-19 17:03:33 +00:00)


In [37]:
#logistic regression after using PCA
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(train_img, train_labels)
# logisticRegr.score(test_img, test_labels)
predictions  = logisticRegr.predict(test_img)
print('Accuracy of Logistic Regression : ', metrics.accuracy_score(test_labels, predictions))

Accuracy of Logistic Regression :  0.916
time: 6.57 s (started: 2022-04-19 17:04:13 +00:00)


# Comparison of the Models

In [38]:
import pandas as pd
data = {'Non PCA':['12 s','8.77 s'],
        'PCA':['36.6 s','4.27 s'],
        'Randomized PCA':['36.9 s','4.4s'],
        'Incr PCA':['39.7 s','6.57 s']
        }
df = pd.DataFrame(data, index =['Random Forest',
                                'Logistic Regression'
                                ])
df.name='Execution Time for different models'
print(df.name)
df

Execution Time for different models


,Non PCA,PCA,Randomized PCA,Incr PCA
Random Forest,12 s,36.6 s,36.9 s,39.7 s
Logistic Regression,8.77 s,4.27 s,4.4s,6.57 s


time: 26 ms (started: 2022-04-19 17:04:53 +00:00)


In [39]:
data = {'Non PCA':[95.85,91.22],
        'PCA':[92.32,91.58],
        'Randomized PCA':[92.53,91.43],
        'Incr PCA':[92.42,91.6]
        }
df = pd.DataFrame(data, index =['Random Forest',
                                'Logistic Regression'
                                ])
df.name='Accuracy for different models'
print(df.name)
df

Accuracy for different models


,Non PCA,PCA,Randomized PCA,Incr PCA
Random Forest,95.85,92.32,92.53,92.42
Logistic Regression,91.22,91.58,91.43,91.60


time: 17.7 ms (started: 2022-04-19 17:04:54 +00:00)
